# Batch Renaming Image files

I have a bunch of pictures that have the time tags incorrect and have different filenames.
My goal here is to correct and standadise these two things.

There are 6 dates to change:
- System dates: windows stores these in UNIX epoch time.
    - "Date accessed"
    - "Date modified"
    - "Date created"
- jpg exif dates: stored according to the exif standard (see below).
    - "Date taken"
    - "Date digitized"
    - "Date original"

https://stackoverflow.com/questions/33031663/how-to-change-image-captured-date-in-python

In [2]:
import os

In [3]:
# https://docs.python.org/3/library/datetime.html
# https://docs.python.org/3/library/datetime.html#datetime.tzinfo
import datetime

# https://pypi.org/project/pytz/
import pytz

In [4]:
# got this module from here 
# https://stackoverflow.com/questions/4996405/how-do-i-change-the-file-creation-date-of-a-windows-file
from win32_setctime import setctime

#### About piexif
- It seems that one cannot edit a single exif metadata entry with piexif
- piexif replaces the whole exif metadata. Be careful!

In [5]:
# https://piexif.readthedocs.io/en/latest/index.html
# https://pypi.org/project/piexif/
import piexif

## Moving between timezones
The following functions help to visualise and change timestamps between different formats and timezones.

We work with Unix epoch time and convert from and to it.

In [6]:
# One should get the offsets with the Time Taken (in Epoch) and tz.
# In this way timezone conversions take Daylight Savings into account.
# Getting the offset in the correct format is inconvenient.
def getOffset(epoch,tz):
    offset = datetime.datetime.fromtimestamp(epoch,tz).strftime("%z")
    exifOffset = (offset[0:3] + ":" + offset[3:5]).encode("utf-8")
    return exifOffset

In [24]:
# epochTime: float
# tz: datetime.tzinfo
def epochToHuman(epochTime,tz): # : str
    humanFormat = "%Y:%m:%d %H:%M:%S.%f  %Z  %z"
    humanDatetime = datetime.datetime.fromtimestamp(epochTime,tz).strftime(humanFormat)
    return humanDatetime


# humanTime: str
# tz: datetime.tzinfo
def humanToEpoch(humanTime,tz): # : float
    humanFormat = "%Y:%m:%d %H:%M:%S.%f"
    timeInTz = tz.localize(datetime.datetime.strptime(humanTime, humanFormat))
    display(timeInTz)
    epochTime = timeInTz.timestamp()
    return epochTime
    

# exifTime == {"datetime": bytes,"subsec": bytes,"offset": bytes}
# tz: datetime.tzinfo
def exifToEpoch(exifTime,tz): # : float
    exifFormat = "%Y:%m:%d %H:%M:%S.%f"
    exifTimeExact = ".".join([exifTime["datetime"].decode(),exifTime["subsec"].decode()])
    dateTime = tz.localize(datetime.datetime.strptime(exifTimeExact, exifFormat))
    epochTime = dateTime.timestamp()
    
    exifOffset = exifTime["offset"]
    if (exifOffset is not None):
        tzOffset = getOffset(epochTime,tz)
        assert exifOffset == tzOffset, f"Different Offsets Warning: Exif offset is {exifOffset}, and tz offset is {tzOffset}"
    
    return epochTime


# epochTime: float
# tz: datetime.tzinfo
def epochToExif(epochTime,tz): # == {"datetime": bytes,"subsec": bytes,"offset": bytes}
    exifFormat = "%Y:%m:%d %H:%M:%S.%f"
    dateTime = datetime.datetime.fromtimestamp(epochTime,tz).strftime(exifFormat)
    (exifDatetime, exifSubsec) = (x.encode("utf-8") for x in dateTime.split("."))
    exifOffset = getOffset(epochTime,tz)
    timeInTz = {"datetime": exifDatetime, "subsec": exifSubsec, "offset": exifOffset}
    return timeInTz


# epochTime: float
# tz: datetime.tzinfo
def epochToFilename(epochTime,tz): # YYmmdd_HHMMSSfff: str
    timeFormat = "%Y%m%d_%H%M%S%f"
    fileDatetime = datetime.datetime.fromtimestamp(epochTime,tz).strftime(timeFormat)
    return fileDatetime[:-3]

Conversions done in Unix Epoch time should not be needed.

## Editing System times

Check when to use each timezone here:

https://www.timeanddate.com/worldclock/converter.html?iso=20180214T105200&p1=279&p2=155&p3=tz_gmt&p4=240&p5=22

https://www.timeanddate.com/time/change/australia/sydney?year=2017

Timezone offsets in "±HH:MM"
- CKT  -10:00 Rarotonga
- CSTE -06:00 Mexico
- CDTE -05:00 Mexico DST
- AEST +10:00 Sydney
- AEDT +11:00 Sydney DST

In [8]:
def extractSysEpochData(file): # {"atime": epoch, "mtime": epoch, "ctime": epoch}
    atime = os.path.getatime(file)
    mtime = os.path.getmtime(file)
    ctime = os.path.getctime(file)
    sysTimeData = {"atime": atime, "mtime": mtime, "ctime": ctime}
    print(f"System time data extracted from file {file} is:"  + "\n" + f"{sysTimeData}")
    return sysTimeData

In [9]:
def minSysEpoch(file): # : epoch
    sysEpochs = extractSysEpochData(file)
    minEpoch = min(sysEpochs.values())
    return minEpoch

In [10]:
def changeSysEpochData(file,epoch):
    os.utime(file, (epoch, epoch)) #(atime, mtime)
    setctime(file, epoch) # ctime
    print(f"Successfully changed {epoch}")

## Exif metadata

Since the 2012 revision exif metadata can now include timezone offset

2012 CIPA Standard https://www.cipa.jp/std/documents/e/DC-008-2012_E.pdf

2019 CIPA Standard https://www.cipa.jp/std/documents/e/DC-X008-Translation-2019-E.pdf

Wiki https://en.wikipedia.org/wiki/Exif

In [12]:
piexif.TAGS

{'Image': {11: {'name': 'ProcessingSoftware', 'type': 2},
  254: {'name': 'NewSubfileType', 'type': 4},
  255: {'name': 'SubfileType', 'type': 3},
  256: {'name': 'ImageWidth', 'type': 4},
  257: {'name': 'ImageLength', 'type': 4},
  258: {'name': 'BitsPerSample', 'type': 3},
  259: {'name': 'Compression', 'type': 3},
  262: {'name': 'PhotometricInterpretation', 'type': 3},
  263: {'name': 'Threshholding', 'type': 3},
  264: {'name': 'CellWidth', 'type': 3},
  265: {'name': 'CellLength', 'type': 3},
  266: {'name': 'FillOrder', 'type': 3},
  269: {'name': 'DocumentName', 'type': 2},
  270: {'name': 'ImageDescription', 'type': 2},
  271: {'name': 'Make', 'type': 2},
  272: {'name': 'Model', 'type': 2},
  273: {'name': 'StripOffsets', 'type': 4},
  274: {'name': 'Orientation', 'type': 3},
  277: {'name': 'SamplesPerPixel', 'type': 3},
  278: {'name': 'RowsPerStrip', 'type': 4},
  279: {'name': 'StripByteCounts', 'type': 4},
  282: {'name': 'XResolution', 'type': 5},
  283: {'name': 'YRes

In [11]:
# (DateTime, SubsecTime, OffsetTime)
# (DateTimeOriginal, SubsecTimeOriginal, OffsetTimeOriginal)
# (DateTimeDigitized, SubsecTimeDigitized, OffsetTimeDigitized)
exifTimeTags = {"edit": (306, 37520, 36880), "orig": (36867, 37521, 36881), "digi": (36868, 37522, 36882)}


In [12]:
def extractExifModel(file):
    exifData = piexif.load(file)
    return exifData["0th"][272] # exif Camera Model Tag is 272

In [13]:
def extractExifTimeData(file): # {"edit: exifTime, "orig": exifTime, "digi": exifTime}
    exifData = piexif.load(file)
    exifTimeData = {"edit": {}, "orig": {}, "digi": {}}
            
    for key,value in exifTimeTags.items():
        if key != "edit":
            exifDatetime = exifData["Exif"].get(value[0])
        else:
            exifDatetime = exifData["0th"].get(value[0])
        exifSubsec = exifData["Exif"].get(value[1])
        exifOffset = exifData["Exif"].get(value[2])
        exifTimeData[key] = {"datetime": exifDatetime, "subsec": exifSubsec, "offset": exifOffset}
    
    print(f"Exif time data extracted from file {file} is:" + "\n" + f"{exifTimeData}")
    return exifTimeData

In [14]:
def minExifTime(file,tz=None):  # exifTime == {"datetime": bytes,"subsec": bytes,"offset": bytes}

    exifTimeData = extractExifTimeData(file)
    
    (minExifDatetime,minExifSubsec) = (min([value[i] for (key, value) in exifTimeData.items() if value[i] is not None]) for i in ("datetime","subsec"))
    
    minTime = {}
    minTime["datetime"] = minExifDatetime
    minTime["subsec"] = minExifSubsec
    
    # All my images have None in the Offset fields.
    # Images that have something in the Offset fields will be handled correctly.
    allOffsets = [value["offset"] for key ,value in exifTimeData.items()]
    
    # If one or more fields have offset != None, then all do and they're all equal to each other.
    assert all(offset == allOffsets[0] for offset in allOffsets) , "Offsets in exif tags do not coincide"
    
    minTime["offset"] = allOffsets[0]
    
    # Furthermore, if tz is given, offset data is extracted from tz instead.
    if tz is not None:
        # exifToEpoch checks that minTime and tz offsets coincide.
        epoch = exifToEpoch(minTime,tz)
        exifOffset = getOffset(epoch,tz)
        minTime["offset"] = exifOffset
    
    return minTime

In [15]:
def changeExifTimeData(file,exifTime):
    exifData = piexif.load(file)
    newExifData = exifData.copy()
    
    for (key, value) in exifTimeTags.items():
        newExifData["Exif"][value[1]] = exifTime["subsec"]
        newExifData["Exif"][value[2]] = exifTime["offset"]
        if key != "edit":
            newExifData["Exif"][value[0]] = exifTime["datetime"]
            print(key, newExifData["Exif"][value[0]], newExifData["Exif"][value[1]], newExifData["Exif"][value[2]])
        else:
            newExifData["0th"][value[0]] = exifTime["datetime"]
            print(key, newExifData["0th"][value[0]], newExifData["Exif"][value[1]], newExifData["Exif"][value[2]])
    
    exifAsBytes = piexif.dump(newExifData)
    piexif.remove(file)
    piexif.insert(exifAsBytes,file)
    print(f"Successfully changed {exifTime}")

## Filenames

Renaming is the last step, we need all time metadata to be consistent.

We remove "atime" from the checks because this changes every time we access the file.

In [16]:
def checkTimeMetadata(file,tz):
   
    exifTimeData = extractExifTimeData(file)
    
    # Remember that calling exifToEpoch checks that offsets correspond to those of the tz
    exifEpochData = {key: exifToEpoch(exifValue,tz) for key, exifValue in exifTimeData.items()}
    print(exifEpochData)
    assert all(exifTime == exifEpochData["edit"] for exifTime in exifEpochData.values()) , "Warning: exif data is not consistent: \n" + f"{exifEpochData}"
    print("Exif Data is correct")
    
    sysEpochData = extractSysEpochData(file)
    assert all(abs(epoch1 - exifEpochData["edit"]) < 1 for key, epoch1 in sysEpochData.items() if key != "atime"), "Warning: system data is not consistent: \n" + f"{sysEpochData}"
    print("System Data is correct")
    
    return exifEpochData["edit"]

In [17]:
imageExtensions = ["jpg","png"]

In [18]:
def changeFilename(file,tz):
    (filename,ext) = file.split(".")
    newExt = ext.lower()
    assert newExt in imageExtensions, f"file {file} is not an jpg or png file"
    epoch = checkTimeMetadata(file,tz)
    newFilename = f"{epochToFilename(epoch,tz)}.{newExt}"
    os.rename(file,newFilename)
    print(f"Successfully changed {newFilename}")

## Batch fixing and renaming

In [19]:
def fixFileTimeData(file):
    (correctExif,correctEpoch,correctTz) = getCorrectTimes(file)
    # Change Exif datetimes.
    changeExifTimeData(file,correctExif)
    # Change system epochs. 
    changeSysEpochData(file,correctEpoch)
    # Change filename
    changeFilename(file,correctTz)

In [20]:
def fixFolderTimeData(path):
    cwd = os.getcwd()
    os.chdir(path)
    files = os.listdir(path)
    images = [file for file in files if file[-3:].lower() in imageExtensions]
    for file in images:
        print("\n")
        try:
            fixFileTimeData(file)
        except:
            continue
    os.chdir(cwd)

If one only wants to change the filenames according to the format given by epochToFilename(epochTime,tz): YYmmdd_HHMMSSfff.

Use the function below.

In [21]:
def changeFolderFilenames(path,tz):
    cwd = os.getcwd()
    os.chdir(path)
    files = os.listdir(path)
    images = [file for file in files if file[-3:].lower() in imageExtensions]
    for file in images:
        changeFilename(file,tz)
        print("\n")
    os.chdir(cwd)

## Fixing file system time tags and filename mess

In 2018 I took pictures with a Canon in Rarotonga, for the first part of the trip, I forgot to adjust the time of the camera and all pictures taken before the StagDo have the following problems:
- All the dates are in Sydney time AEDT; but
- Pictures taken in Sydney have "minDate" and "Date Taken" are off by 1hr. 
    - At first I thought only exif dates were the correct Sydney time.
    - I have now realised that windows takes daylight savings time into account when showing system dates.
    - At this point I am still not sure which is the correct UNIX time to write.
- filenames start with "_MG_" or "IMG_" followed by a number. Numbers are in sequential order of photos taken by the camera.

Just before the StagDo I must have adjusted the camera time, but I set it up am instead of pm. So pictures during the StagDo and after, are 12 hours off.
- I am preteding this pictures are in Madrid time.

So the solution should:
- Capture "Date Taken" with nanoseconds.
- if (picture.BeforeStag) 
    - if picture.inSydney "all dates" == "Date Taken"
    - if picture.inRaro "all dates" == "Date Taken" - 20h
- if (picture.notBeforeStag) "all dates" == "Date Taken" + 12h

Files taken with Motorola have the following problem:
- "Date modified" is 1hr earlier than "Date Taken"
- The correct time for those pictures is: "Date Taken" (?)

## How to use this script

You only need to write the function "getCorrectTimes" whose input is the name of the file as a string and output a triple (camExif,camEpoch,camTz)
- camExif == {"datetime": bytes,"subsec": bytes,"offset": bytes}
- camEpoch: float 
- camTz: datetime.tzinfo

Finally you should run fixFileTimeData or fixFolderTimeData or changeFolderFilenames

The code below will work only for my particular problem

In [114]:
display(pytz.all_timezones_set)

{'Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [22]:
# constants in datetime.tzinfo
tzRar = pytz.timezone("Pacific/Rarotonga")
tzSyd = pytz.timezone("Australia/Sydney")
tzMex = pytz.timezone("America/Mexico_City")
tzMad = pytz.timezone("Europe/Madrid")
# tzTok = pytz.timezone("Asia/Tokyo")
# tzAEDT = pytz.timezone("Etc/GMT-11")
# tzAEST = pytz.timezone("Etc/GMT-10")

In [30]:
# Relevant dates as epochs
depSyd = "2018:08:14 11:50:00"
arrSyd = "2018:08:22 09:00:00"
depSydEpoch = humanToEpoch(depSyd+".0",tzSyd)
arrSydEpoch = humanToEpoch(arrSyd+".0",tzSyd)

depMex2018 = "2018:01:18 00:00:00"
arrMex2018 = "2018:12:10 20:40:00"

datetime.datetime(2018, 8, 14, 11, 50, tzinfo=<DstTzInfo 'Australia/Sydney' AEST+10:00:00 STD>)

datetime.datetime(2018, 8, 22, 9, 0, tzinfo=<DstTzInfo 'Australia/Sydney' AEST+10:00:00 STD>)

In [31]:
# Output should be a triple (camExif,camEpoch,camTz)
# camExif == {"datetime": bytes,"subsec": bytes,"offset": bytes}
# camEpoch: float 
# camTz: datetime.tzinfo
def getCorrectTimes(file):
    
    fileTimeExif = minExifTime(file)
    print(">>Min exif time extracted:\n" + f"{fileTimeExif}")
    
    fileEpochSys = minSysEpoch(file)
    print(">>Min system epoch extracted:\n" + f"{fileEpochSys}")
    
    print(">>Min system epoch extracted in Sydney time:\n" + f"{epochToHuman(fileEpochSys,tzSyd)}")
    # print("Min system epoch extracted local:\n" + f"{epochToHuman(fileEpochSys,tzSyd)}")
    
    camModel = extractExifModel(file)
    print(f">>Picture taken with {camModel}")
    
    # Process images taken with the Canon
    if camModel == b'Canon EOS DIGITAL REBEL XS':
        # if the picture has been already edited,
        # exifToEpoch will fail to match the offsets.
        
        if file[4:8] < "8293": #IMG_XXXX.JPG
            correctEpoch = exifToEpoch(fileTimeExif,tzSyd)
            print(f">>Correct Unix Epoch:\n" + f"{correctEpoch}")
        elif "8292" < file[4:8]: # and file[4:8] < "8454" ???
            correctEpoch = exifToEpoch(fileTimeExif,tzMad) + 24*60*60
            print(f">>Correct Unix Epoch:\n" + f"{correctEpoch}")
        
        if depSydEpoch < correctEpoch and correctEpoch < arrSydEpoch:
            print(">>In Rarotonga")
            correctTz = tzRar
        else:
            print(">>In Sydney")
            correctTz = tzSyd
        
        print(f">>Correct time:\n" + f"{epochToHuman(correctEpoch,correctTz)}")
        print(">>Correct epoch and time Windows should show:\n" + f"{correctEpoch}\n" + f"{epochToHuman(correctEpoch,tzSyd)}")
    
        correctExif = epochToExif(correctEpoch,correctTz)
        print(">>Correct exif time data:\n" + f"{correctExif}")
        
        return (correctExif,correctEpoch,correctTz)
    
    # Process images taken with the cellphone
    elif camModel == b'MotoG3':
        
        if depSyd < fileTimeExif["datetime"].decode() and fileTimeExif["datetime"].decode() < arrSyd:
            print(">>In Rarotonga")
            correctTz = tzRar
        elif depMex2018 < fileTimeExif["datetime"].decode() and fileTimeExif["datetime"].decode() < arrMex2018:
            print(">>In Sydney")
            correctTz = tzSyd
        else:
            print(">>In Mexico")
            correctTz = tzMex
        
        correctEpoch = exifToEpoch(fileTimeExif,correctTz)
        
        print(f">>Correct time:\n" + f"{epochToHuman(correctEpoch,correctTz)}")
        print(">>Correct epoch and time Windows should show:\n" + f"{correctEpoch}\n" + f"{epochToHuman(correctEpoch,tzSyd)}")
    
        correctExif = epochToExif(correctEpoch,correctTz)
        print(">>Correct exif time data:\n" + f"{correctExif}")
        
        return (correctExif,correctEpoch,correctTz)
    
    # Other pictures are not processed
    else:
        print(f">>File {file} was taken with {camModel}; not Canon or Moto G3")
    

## Actually Fixing everything!

In [163]:
fixFolderTimeData(r"F:\Pictures\Fotos\2018\08 Rarotonga\RaroTimeCopy")



Exif time data extracted from file IMG_8141.JPG is:
{'edit': {'datetime': b'2018:08:14 21:52:37', 'subsec': b'71', 'offset': None}, 'orig': {'datetime': b'2018:08:14 21:52:37', 'subsec': b'71', 'offset': None}, 'digi': {'datetime': b'2018:08:14 21:52:37', 'subsec': b'71', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:14 21:52:37', 'subsec': b'71', 'offset': None}
System time data extracted from file IMG_8141.JPG is:
{'atime': 1643939912.0612097, 'mtime': 1534243958.0, 'ctime': 1643939790.7261584}
>>Min system epoch extracted:
1534243958.0
>>Min system epoch extracted in Sydney time:
2018/08/14 20:52:38.000000  AEST  +1000
>>Picture taken with b'Canon EOS DIGITAL REBEL XS'
>>Correct Unix Epoch:
1534247557.71
>>In Rarotonga
>>Correct time:
2018/08/14 01:52:37.710000  -10  -1000
>>Correct epoch and time Windows should show:
1534247557.71
2018/08/14 21:52:37.710000  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:14 01:52:37', 'subsec': b'710000', 'offse

In [165]:
# Exif time data extracted from file _MG_8414.JPG is:
# {'edit': {'datetime': b'2018:08:19 23:54:49', 'subsec': b'03', 'offset': None}, 'orig': {'datetime': b'2018:08:19 23:54:49', 'subsec': b'03', 'offset': None}, 'digi': {'datetime': b'2018:08:19 23:54:49', 'subsec': b'03', 'offset': None}}
# >>Min exif time extracted:
# {'datetime': b'2018:08:19 23:54:49', 'subsec': b'03', 'offset': None}
# System time data extracted from file _MG_8414.JPG is:
# {'atime': 1643940002.3872237, 'mtime': 1534683290.0, 'ctime': 1643939783.914441}
# >>Min system epoch extracted:
# 1534683290.0
# >>Min system epoch extracted in Sydney time:
# 2018/08/19 22:54:50.000000  AEST  +1000
# >>Picture taken with b'Canon EOS DIGITAL REBEL XS'
# >>Correct Unix Epoch:
# 1534802089.03
# >>In Rarotonga
# >>Correct time:
# 2018/08/20 11:54:49.030000  -10  -1000
# >>Correct epoch and time Windows should show:
# 1534802089.03
# 2018/08/21 07:54:49.030000  AEST  +1000
# >>Correct exif time data:
# {'datetime': b'2018:08:20 11:54:49', 'subsec': b'030000', 'offset': b'-10:00'}
# edit b'2018:08:20 11:54:49' b'030000' b'-10:00'
# orig b'2018:08:20 11:54:49' b'030000' b'-10:00'
# digi b'2018:08:20 11:54:49' b'030000' b'-10:00'

cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\08 Rarotonga\RaroTimeCopy")
file = "_MG_8414.JPG"
correctExif = {'datetime': b'2018:08:20 11:54:49', 'subsec': b'030000', 'offset': b'-10:00'}
correctEpoch = 1534802089.03
correctTz = tzRar
# Change Exif datetimes.
changeExifTimeData(file,correctExif)
# Change system epochs. 
changeSysEpochData(file,correctEpoch)
# Change filename
changeFilename(file,correctTz)

os.chdir(cwd)

edit b'2018:08:20 11:54:49' b'030000' b'-10:00'
orig b'2018:08:20 11:54:49' b'030000' b'-10:00'
digi b'2018:08:20 11:54:49' b'030000' b'-10:00'
Successfully changed {'datetime': b'2018:08:20 11:54:49', 'subsec': b'030000', 'offset': b'-10:00'}
Successfully changed 1534802089.03
Exif time data extracted from file _MG_8414.JPG is:
{'edit': {'datetime': b'2018:08:20 11:54:49', 'subsec': b'030000', 'offset': b'-10:00'}, 'orig': {'datetime': b'2018:08:20 11:54:49', 'subsec': b'030000', 'offset': b'-10:00'}, 'digi': {'datetime': b'2018:08:20 11:54:49', 'subsec': b'030000', 'offset': b'-10:00'}}
{'edit': 1534802089.03, 'orig': 1534802089.03, 'digi': 1534802089.03}
Exif Data is correct
System time data extracted from file _MG_8414.JPG is:
{'atime': 1643940249.5796123, 'mtime': 1534802089.03, 'ctime': 1534802089.03}
System Data is correct
Successfully changed 20180820_115449030.jpg


For some reason, three weren't fixed. I just manually fixed them below.

In [143]:
# Exif time data extracted from file _MG_8400.JPG is:
# {'edit': {'datetime': b'2018:08:19 23:38:14', 'subsec': b'83', 'offset': None}, 'orig': {'datetime': b'2018:08:19 23:38:14', 'subsec': b'83', 'offset': None}, 'digi': {'datetime': b'2018:08:19 23:38:14', 'subsec': b'83', 'offset': None}}
# >>Min exif time extracted:
# {'datetime': b'2018:08:19 23:38:14', 'subsec': b'83', 'offset': None}
# System time data extracted from file _MG_8400.JPG is:
# {'atime': 1643934896.748373, 'mtime': 1534682294.0, 'ctime': 1643934727.6914282}
# >>Min system epoch extracted:
# 1534682294.0
# >>Min system epoch extracted in Sydney time:
# 2018/08/19 22:38:14.000000  AEST  +1000
# >>Picture taken with b'Canon EOS DIGITAL REBEL XS'
# >>Correct Unix Epoch:
# 1534714694.83
# >>In Rarotonga
# >>Correct time:
# 2018/08/19 11:38:14.830000  -10  -1000
# >>Correct epoch and time Windows should show:
# 1534714694.83
# 2018/08/20 07:38:14.830000  AEST  +1000
# >>Correct exif time data:
# {'datetime': b'2018:08:19 11:38:14', 'subsec': b'830000', 'offset': b'-10:00'}
# edit b'2018:08:19 11:38:14' b'830000' b'-10:00'
# orig b'2018:08:19 11:38:14' b'830000' b'-10:00'
# digi b'2018:08:19 11:38:14' b'830000' b'-10:00'


cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\08 Rarotonga\RaroTimeCopy3")
file = "_MG_8400.JPG"
correctExif = {'datetime': b'2018:08:19 11:38:14', 'subsec': b'830000', 'offset': b'-10:00'}
correctEpoch = 1534714694.83
correctTz = tzRar
# Change Exif datetimes.
changeExifTimeData(file,correctExif)
# Change system epochs. 
changeSysEpochData(file,correctEpoch)
# Change filename
changeFilename(file,correctTz)

os.chdir(cwd)


edit b'2018:08:19 11:38:14' b'830000' b'-10:00'
orig b'2018:08:19 11:38:14' b'830000' b'-10:00'
digi b'2018:08:19 11:38:14' b'830000' b'-10:00'
Successfully changed {'datetime': b'2018:08:19 11:38:14', 'subsec': b'830000', 'offset': b'-10:00'}
Successfully changed 1534714694.83
Exif time data extracted from file _MG_8400.JPG is:
{'edit': {'datetime': b'2018:08:19 11:38:14', 'subsec': b'830000', 'offset': b'-10:00'}, 'orig': {'datetime': b'2018:08:19 11:38:14', 'subsec': b'830000', 'offset': b'-10:00'}, 'digi': {'datetime': b'2018:08:19 11:38:14', 'subsec': b'830000', 'offset': b'-10:00'}}
{'edit': 1534714694.83, 'orig': 1534714694.83, 'digi': 1534714694.83}
Exif Data is correct
System time data extracted from file _MG_8400.JPG is:
{'atime': 1643936187.732864, 'mtime': 1534714694.83, 'ctime': 1534714694.83}
System Data is correct
Successfully changed 20180819_113814830.jpg


In [144]:
# Exif time data extracted from file _MG_8407.JPG is:
# {'edit': {'datetime': b'2018:08:19 23:40:56', 'subsec': b'03', 'offset': None}, 'orig': {'datetime': b'2018:08:19 23:40:56', 'subsec': b'03', 'offset': None}, 'digi': {'datetime': b'2018:08:19 23:40:56', 'subsec': b'03', 'offset': None}}
# >>Min exif time extracted:
# {'datetime': b'2018:08:19 23:40:56', 'subsec': b'03', 'offset': None}
# System time data extracted from file _MG_8407.JPG is:
# {'atime': 1643934899.5400932, 'mtime': 1534682456.0, 'ctime': 1643934728.4869554}
# >>Min system epoch extracted:
# 1534682456.0
# >>Min system epoch extracted in Sydney time:
# 2018/08/19 22:40:56.000000  AEST  +1000
# >>Picture taken with b'Canon EOS DIGITAL REBEL XS'
# >>Correct Unix Epoch:
# 1534714856.03
# >>In Rarotonga
# >>Correct time:
# 2018/08/19 11:40:56.030000  -10  -1000
# >>Correct epoch and time Windows should show:
# 1534714856.03
# 2018/08/20 07:40:56.030000  AEST  +1000
# >>Correct exif time data:
# {'datetime': b'2018:08:19 11:40:56', 'subsec': b'030000', 'offset': b'-10:00'}
# edit b'2018:08:19 11:40:56' b'030000' b'-10:00'
# orig b'2018:08:19 11:40:56' b'030000' b'-10:00'
# digi b'2018:08:19 11:40:56' b'030000' b'-10:00'


cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\08 Rarotonga\RaroTimeCopy3")
file = "_MG_8407.JPG"
correctExif = {'datetime': b'2018:08:19 11:40:56', 'subsec': b'030000', 'offset': b'-10:00'}
correctEpoch = 1534714856.03
correctTz = tzRar
# Change Exif datetimes.
changeExifTimeData(file,correctExif)
# Change system epochs. 
changeSysEpochData(file,correctEpoch)
# Change filename
changeFilename(file,correctTz)

os.chdir(cwd)

edit b'2018:08:19 11:40:56' b'030000' b'-10:00'
orig b'2018:08:19 11:40:56' b'030000' b'-10:00'
digi b'2018:08:19 11:40:56' b'030000' b'-10:00'
Successfully changed {'datetime': b'2018:08:19 11:40:56', 'subsec': b'030000', 'offset': b'-10:00'}
Successfully changed 1534714856.03
Exif time data extracted from file _MG_8407.JPG is:
{'edit': {'datetime': b'2018:08:19 11:40:56', 'subsec': b'030000', 'offset': b'-10:00'}, 'orig': {'datetime': b'2018:08:19 11:40:56', 'subsec': b'030000', 'offset': b'-10:00'}, 'digi': {'datetime': b'2018:08:19 11:40:56', 'subsec': b'030000', 'offset': b'-10:00'}}
{'edit': 1534714856.03, 'orig': 1534714856.03, 'digi': 1534714856.03}
Exif Data is correct
System time data extracted from file _MG_8407.JPG is:
{'atime': 1643936304.963228, 'mtime': 1534714856.03, 'ctime': 1534714856.03}
System Data is correct
Successfully changed 20180819_114056030.jpg


In [146]:
# Exif time data extracted from file _MG_8445.JPG is:
# {'edit': {'datetime': b'2018:08:20 00:14:51', 'subsec': b'52', 'offset': None}, 'orig': {'datetime': b'2018:08:20 00:14:51', 'subsec': b'52', 'offset': None}, 'digi': {'datetime': b'2018:08:20 00:14:51', 'subsec': b'52', 'offset': None}}
# >>Min exif time extracted:
# {'datetime': b'2018:08:20 00:14:51', 'subsec': b'52', 'offset': None}
# System time data extracted from file _MG_8445.JPG is:
# {'atime': 1643934920.319486, 'mtime': 1534684492.0, 'ctime': 1643934730.110069}
# >>Min system epoch extracted:
# 1534684492.0
# >>Min system epoch extracted in Sydney time:
# 2018/08/19 23:14:52.000000  AEST  +1000
# >>Picture taken with b'Canon EOS DIGITAL REBEL XS'
# >>Correct Unix Epoch:
# 1534716891.52
# >>In Rarotonga
# >>Correct time:
# 2018/08/19 12:14:51.520000  -10  -1000
# >>Correct epoch and time Windows should show:
# 1534716891.52
# 2018/08/20 08:14:51.520000  AEST  +1000
# >>Correct exif time data:
# {'datetime': b'2018:08:19 12:14:51', 'subsec': b'520000', 'offset': b'-10:00'}
# edit b'2018:08:19 12:14:51' b'520000' b'-10:00'
# orig b'2018:08:19 12:14:51' b'520000' b'-10:00'
# digi b'2018:08:19 12:14:51' b'520000' b'-10:00'

cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\08 Rarotonga\RaroTimeCopy3")
file = "_MG_8445.JPG"
correctExif = {'datetime': b'2018:08:19 12:14:51', 'subsec': b'520000', 'offset': b'-10:00'}
correctEpoch = 1534716891.52
correctTz = tzRar
# Change Exif datetimes.
changeExifTimeData(file,correctExif)
# Change system epochs. 
changeSysEpochData(file,correctEpoch)
# Change filename
changeFilename(file,correctTz)

os.chdir(cwd)

edit b'2018:08:19 12:14:51' b'520000' b'-10:00'
orig b'2018:08:19 12:14:51' b'520000' b'-10:00'
digi b'2018:08:19 12:14:51' b'520000' b'-10:00'
Successfully changed {'datetime': b'2018:08:19 12:14:51', 'subsec': b'520000', 'offset': b'-10:00'}
Successfully changed 1534716891.52
Exif time data extracted from file _MG_8445.JPG is:
{'edit': {'datetime': b'2018:08:19 12:14:51', 'subsec': b'520000', 'offset': b'-10:00'}, 'orig': {'datetime': b'2018:08:19 12:14:51', 'subsec': b'520000', 'offset': b'-10:00'}, 'digi': {'datetime': b'2018:08:19 12:14:51', 'subsec': b'520000', 'offset': b'-10:00'}}
{'edit': 1534716891.52, 'orig': 1534716891.52, 'digi': 1534716891.52}
Exif Data is correct
System time data extracted from file _MG_8445.JPG is:
{'atime': 1643936404.7509034, 'mtime': 1534716891.52, 'ctime': 1534716891.52}
System Data is correct
Successfully changed 20180819_121451520.jpg


In [147]:
fixFolderTimeData(r"F:\Pictures\Fotos\2018\08 Rarotonga\SydneyTimeCopy")



Exif time data extracted from file IMG_8136.JPG is:
{'edit': {'datetime': b'2018:08:14 06:05:12', 'subsec': b'04', 'offset': None}, 'orig': {'datetime': b'2018:08:14 06:05:12', 'subsec': b'04', 'offset': None}, 'digi': {'datetime': b'2018:08:14 06:05:12', 'subsec': b'04', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:14 06:05:12', 'subsec': b'04', 'offset': None}
System time data extracted from file IMG_8136.JPG is:
{'atime': 1643936699.2817545, 'mtime': 1534187114.0, 'ctime': 1643936664.900521}
>>Min system epoch extracted:
1534187114.0
>>Min system epoch extracted in Sydney time:
2018/08/14 05:05:14.000000  AEST  +1000
>>Picture taken with b'Canon EOS DIGITAL REBEL XS'
>>Correct Unix Epoch:
1534190712.04
>>In Sydney
>>Correct time:
2018/08/14 06:05:12.040000  AEST  +1000
>>Correct epoch and time Windows should show:
1534190712.04
2018/08/14 06:05:12.040000  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:14 06:05:12', 'subsec': b'040000', 'offset':

In [152]:
fixFolderTimeData(r"F:\Pictures\Fotos\2018\08 Rarotonga\MotorolaCopy")



Exif time data extracted from file IMG_20180814_053540.jpg is:
{'edit': {'datetime': b'2018:08:14 05:35:44', 'subsec': b'075306', 'offset': None}, 'orig': {'datetime': b'2018:08:14 05:35:44', 'subsec': b'075306', 'offset': None}, 'digi': {'datetime': b'2018:08:14 05:35:44', 'subsec': b'075306', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:14 05:35:44', 'subsec': b'075306', 'offset': None}
System time data extracted from file IMG_20180814_053540.jpg is:
{'atime': 1643937444.809515, 'mtime': 1534185344.0, 'ctime': 1643936723.546019}
>>Min system epoch extracted:
1534185344.0
>>Min system epoch extracted in Sydney time:
2018/08/14 04:35:44.000000  AEST  +1000
>>Picture taken with b'MotoG3'
>>In Sydney
>>Correct time:
2018/08/14 05:35:44.075306  AEST  +1000
>>Correct epoch and time Windows should show:
1534188944.075306
2018/08/14 05:35:44.075306  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:14 05:35:44', 'subsec': b'075306', 'offset': b'+10:00'}
edi

In [28]:
fixFolderTimeData(r"F:\Pictures\Fotos\2018\Canon")



Exif time data extracted from file _MG_8110.JPG is:
{'edit': {'datetime': b'2018:08:01 23:27:11', 'subsec': b'03', 'offset': None}, 'orig': {'datetime': b'2018:08:01 23:27:11', 'subsec': b'03', 'offset': None}, 'digi': {'datetime': b'2018:08:01 23:27:11', 'subsec': b'03', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:01 23:27:11', 'subsec': b'03', 'offset': None}
System time data extracted from file _MG_8110.JPG is:
{'atime': 1643942021.0055616, 'mtime': 1533126432.0, 'ctime': 1533126430.0}
>>Min system epoch extracted:
1533126430.0
>>Min system epoch extracted in Sydney time:
2018:08:01 22:27:10.000000  AEST  +1000
>>Picture taken with b'Canon EOS DIGITAL REBEL XS'
>>Correct Unix Epoch:
1533130031.03
>>In Sydney
>>Correct time:
2018:08:01 23:27:11.030000  AEST  +1000
>>Correct epoch and time Windows should show:
1533130031.03
2018:08:01 23:27:11.030000  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:01 23:27:11', 'subsec': b'030000', 'offset': b'+1

In [32]:
fixFolderTimeData(r"F:\Pictures\Fotos\2018\MotoG3Copy")



Exif time data extracted from file IMG_20180117_163358.jpg is:
{'edit': {'datetime': b'2018:01:17 16:33:58', 'subsec': b'864195', 'offset': None}, 'orig': {'datetime': b'2018:01:17 16:33:58', 'subsec': b'864195', 'offset': None}, 'digi': {'datetime': b'2018:01:17 16:33:58', 'subsec': b'864195', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:01:17 16:33:58', 'subsec': b'864195', 'offset': None}
System time data extracted from file IMG_20180117_163358.jpg is:
{'atime': 1643943583.9358764, 'mtime': 1516228439.0, 'ctime': 1643943410.341484}
>>Min system epoch extracted:
1516228439.0
>>Min system epoch extracted in Sydney time:
2018:01:18 09:33:59.000000  AEDT  +1100
>>Picture taken with b'MotoG3'
>>In Mexico
>>Correct time:
2018:01:17 16:33:58.864195  CST  -0600
>>Correct epoch and time Windows should show:
1516228438.864195
2018:01:18 09:33:58.864195  AEDT  +1100
>>Correct exif time data:
{'datetime': b'2018:01:17 16:33:58', 'subsec': b'864195', 'offset': b'-06:00'}
edi

In [34]:
fixFolderTimeData(r"F:\Pictures\Fotos\2018\MotoG3Copy\Retry")



Exif time data extracted from file IMG_20180309_225413.jpg is:
{'edit': {'datetime': None, 'subsec': None, 'offset': None}, 'orig': {'datetime': None, 'subsec': None, 'offset': None}, 'digi': {'datetime': None, 'subsec': None, 'offset': None}}


Exif time data extracted from file IMG_20180516_144754.jpg is:
{'edit': {'datetime': b'2018:05:16 14:47:54', 'subsec': None, 'offset': None}, 'orig': {'datetime': None, 'subsec': None, 'offset': None}, 'digi': {'datetime': None, 'subsec': None, 'offset': None}}


Exif time data extracted from file IMG_20180516_144757.jpg is:
{'edit': {'datetime': b'2018:05:16 14:47:57', 'subsec': None, 'offset': None}, 'orig': {'datetime': None, 'subsec': None, 'offset': None}, 'digi': {'datetime': None, 'subsec': None, 'offset': None}}


In [36]:
# Exif time data extracted from file IMG_20180309_225413.jpg is:
# {'edit': {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': None}, 'orig': {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': None}, 'digi': {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': None}}
# >>Min exif time extracted:
# {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': None}
# System time data extracted from file IMG_20180309_225413.jpg is:
# {'atime': 1643943610.1024144, 'mtime': 1520596452.0, 'ctime': 1643943420.9295037}
# >>Min system epoch extracted:
# 1520596452.0
# >>Min system epoch extracted in Sydney time:
# 2018:03:09 22:54:12.000000  AEDT  +1100
# >>Picture taken with b'MotoG3'
# >>In Sydney
# >>Correct time:
# 2018:03:09 22:54:13.589562  AEDT  +1100
# >>Correct epoch and time Windows should show:
# 1520596453.589562
# 2018:03:09 22:54:13.589562  AEDT  +1100
# >>Correct exif time data:
# {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': b'+11:00'}
# edit b'2018:03:09 22:54:13' b'589562' b'+11:00'
# orig b'2018:03:09 22:54:13' b'589562' b'+11:00'
# digi b'2018:03:09 22:54:13' b'589562' b'+11:00'

cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\MotoG3Copy\Retry")
file = "IMG_20180309_225413.jpg"
correctExif = {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': b"+11:00"}
correctEpoch = 1520596453.589562
correctTz = tzSyd
# Change Exif datetimes.
changeExifTimeData(file,correctExif)
# Change system epochs. 
changeSysEpochData(file,correctEpoch)
# Change filename
changeFilename(file,correctTz)

os.chdir(cwd)

edit b'2018:03:09 22:54:13' b'589562' b'+11:00'
orig b'2018:03:09 22:54:13' b'589562' b'+11:00'
digi b'2018:03:09 22:54:13' b'589562' b'+11:00'
Successfully changed {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': b'+11:00'}
Successfully changed 1520596453.589562
Exif time data extracted from file IMG_20180309_225413.jpg is:
{'edit': {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': b'+11:00'}, 'orig': {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': b'+11:00'}, 'digi': {'datetime': b'2018:03:09 22:54:13', 'subsec': b'589562', 'offset': b'+11:00'}}
{'edit': 1520596453.589562, 'orig': 1520596453.589562, 'digi': 1520596453.589562}
Exif Data is correct
System time data extracted from file IMG_20180309_225413.jpg is:
{'atime': 1643944561.2578769, 'mtime': 1520596453.589562, 'ctime': 1520596453.589562}
System Data is correct
Successfully changed 20180309_225413589.jpg


In [38]:
# Exif time data extracted from file IMG_20180516_144754.jpg is:
# {'edit': {'datetime': b'2018:05:16 14:47:54', 'subsec': None, 'offset': None}, 'orig': {'datetime': None, 'subsec': None, 'offset': None}, 'digi': {'datetime': None, 'subsec': None, 'offset': None}}

cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\MotoG3Copy\Retry")
file = "IMG_20180516_144754.jpg"
correctExif = {'datetime': b'2018:05:16 14:47:54', 'subsec': b'000000', 'offset': b'+10:00'}
correctEpoch = exifToEpoch(correctExif,tzSyd)
correctTz = tzSyd
# Change Exif datetimes.
changeExifTimeData(file,correctExif)
# Change system epochs. 
changeSysEpochData(file,correctEpoch)
# Change filename
changeFilename(file,correctTz)

os.chdir(cwd)

edit b'2018:05:16 14:47:54' b'000000' b'+10:00'
orig b'2018:05:16 14:47:54' b'000000' b'+10:00'
digi b'2018:05:16 14:47:54' b'000000' b'+10:00'
Successfully changed {'datetime': b'2018:05:16 14:47:54', 'subsec': b'000000', 'offset': b'+10:00'}
Successfully changed 1526446074.0
Exif time data extracted from file IMG_20180516_144754.jpg is:
{'edit': {'datetime': b'2018:05:16 14:47:54', 'subsec': b'000000', 'offset': b'+10:00'}, 'orig': {'datetime': b'2018:05:16 14:47:54', 'subsec': b'000000', 'offset': b'+10:00'}, 'digi': {'datetime': b'2018:05:16 14:47:54', 'subsec': b'000000', 'offset': b'+10:00'}}
{'edit': 1526446074.0, 'orig': 1526446074.0, 'digi': 1526446074.0}
Exif Data is correct
System time data extracted from file IMG_20180516_144754.jpg is:
{'atime': 1643944825.928534, 'mtime': 1526446074.0, 'ctime': 1526446074.0}
System Data is correct
Successfully changed 20180516_144754000.jpg


In [40]:
# Exif time data extracted from file IMG_20180516_144757.jpg is:
# {'edit': {'datetime': b'2018:05:16 14:47:57', 'subsec': None, 'offset': None}, 'orig': {'datetime': None, 'subsec': None, 'offset': None}, 'digi': {'datetime': None, 'subsec': None, 'offset': None}}

cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\MotoG3Copy\Retry")
file = "IMG_20180516_144757.jpg"
correctExif = {'datetime': b'2018:05:16 14:47:57', 'subsec': b'000000', 'offset': b'+10:00'}
correctEpoch = exifToEpoch(correctExif,tzSyd)
correctTz = tzSyd
# Change Exif datetimes.
changeExifTimeData(file,correctExif)
# Change system epochs. 
changeSysEpochData(file,correctEpoch)
# Change filename
changeFilename(file,correctTz)

os.chdir(cwd)

edit b'2018:05:16 14:47:57' b'000000' b'+10:00'
orig b'2018:05:16 14:47:57' b'000000' b'+10:00'
digi b'2018:05:16 14:47:57' b'000000' b'+10:00'
Successfully changed {'datetime': b'2018:05:16 14:47:57', 'subsec': b'000000', 'offset': b'+10:00'}
Successfully changed 1526446077.0
Exif time data extracted from file IMG_20180516_144757.jpg is:
{'edit': {'datetime': b'2018:05:16 14:47:57', 'subsec': b'000000', 'offset': b'+10:00'}, 'orig': {'datetime': b'2018:05:16 14:47:57', 'subsec': b'000000', 'offset': b'+10:00'}, 'digi': {'datetime': b'2018:05:16 14:47:57', 'subsec': b'000000', 'offset': b'+10:00'}}
{'edit': 1526446077.0, 'orig': 1526446077.0, 'digi': 1526446077.0}
Exif Data is correct
System time data extracted from file IMG_20180516_144757.jpg is:
{'atime': 1643944839.5642674, 'mtime': 1526446077.0, 'ctime': 1526446077.0}
System Data is correct
Successfully changed 20180516_144757000.jpg


In [161]:
cwd = os.getcwd()
os.chdir(r"F:\Pictures\Fotos\2018\08 Rarotonga\MotorolaCopy")
changeExifTimeData("IMG_20180819_112745.jpg",{'datetime': b'2018:08:19 11:27:45', 'subsec': b"0", 'offset': b'-10:00'})
fixFileTimeData("IMG_20180819_112745.jpg")
os.chdir(cwd)



edit b'2018:08:19 11:27:45' b'0' b'-10:00'
orig b'2018:08:19 11:27:45' b'0' b'-10:00'
digi b'2018:08:19 11:27:45' b'0' b'-10:00'
Successfully changed {'datetime': b'2018:08:19 11:27:45', 'subsec': b'0', 'offset': b'-10:00'}
Exif time data extracted from file IMG_20180819_112745.jpg is:
{'edit': {'datetime': b'2018:08:19 11:27:45', 'subsec': b'0', 'offset': b'-10:00'}, 'orig': {'datetime': b'2018:08:19 11:27:45', 'subsec': b'0', 'offset': b'-10:00'}, 'digi': {'datetime': b'2018:08:19 11:27:45', 'subsec': b'0', 'offset': b'-10:00'}}
>>Min exif time extracted:
{'datetime': b'2018:08:19 11:27:45', 'subsec': b'0', 'offset': b'-10:00'}
System time data extracted from file IMG_20180819_112745.jpg is:
{'atime': 1643938346.1006076, 'mtime': 1643938346.0849836, 'ctime': 1643936733.9179795}
>>Min system epoch extracted:
1643936733.9179795
>>Min system epoch extracted in Sydney time:
2022/02/04 12:05:33.917979  AEDT  +1100
>>Picture taken with b'MotoG3'
>>In Rarotonga
>>Correct time:
2018/08/19 11

In [103]:
def replace(path,toReplace,new = ""):
    cwd = os.getcwd()
    os.chdir(path)
    files = os.listdir(path)
    for filename in files:
        try:
            newFilename = filename.replace(toReplace,new)
            os.rename(filename,newFilename)
        except:
            print(f"file {file} does not contain string {toReplace}")
            continue
    os.chdir(cwd)

In [104]:
pathRaroCopy = r"F:\Pictures\Fotos\2018\08 Rarotonga\RaroTimeCopy2"

replace(pathRaroCopy,"_MG","IMG")

## Pictures in folder to test properties

I first created the "test" function below to be able to figure out what was the problem with all the inconsistent timestamps.
This later evolved into the function "getCorrectTimes".

In [29]:
def test(filex,tz):
    display(os.stat(filex))
    timex = minTime(filex)
    display("minEpochFile: " + str(timex))
    display("minHumanFile: " + str(epochToHuman(timex,tz)))
    timeExifx = minExifTime(filex)
    display("minExifTime: " + str(timeExifx))
    minEpochExif = exifToEpoch(timeExifx,tz)
    display("minEpochExif: " + str(epochToHuman(minEpochExif,tzRar)))
    minExifTime_AU = exifToEpoch(timeExifx,tzSyd)
    display("minExifTime_AU: " + str(epochToHuman(minExifTime_AU,tzSyd)))
    display("Canon to RARO: " + str(epochToHuman(epochAESTtoCKT(minEpochExif),tzRar)))
    imagex = Image.open(filex)
    display(imagex._getexif())
    imagex.close()

In [30]:
# arriving to Rarotonga (Canon)
# test("IMG_8144.jpg",tzRar)
getCorrectTimes("IMG_8144.jpg")

Exif time data extrcted from file IMG_8144.jpg is:
{'edit': {'datetime': b'2018:08:14 21:52:56', 'subsec': b'75', 'offset': None}, 'orig': {'datetime': b'2018:08:14 21:52:56', 'subsec': b'75', 'offset': None}, 'digi': {'datetime': b'2018:08:14 21:52:56', 'subsec': b'75', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:14 21:52:56', 'subsec': b'75', 'offset': None}
System time data extrcted from file IMG_8144.jpg is:
{'atime': 1643866331.263562, 'mtime': 1534243976.0, 'ctime': 1643348298.0901654}
>>Min system epoch extracted:
1534243976.0
>>Min system epoch extracted in Sydney time:
2018/08/14 20:52:56.000000  AEST  +1000
>>Picture taken with b'Canon EOS DIGITAL REBEL XS'
>>Correct Unix Epoch:
1534247576.75
>>In Rarotonga
>>Correct time:
2018/08/14 01:52:56.750000  -10  -1000
>>Correct epoch and time Windows should show:
1534247576.75
2018/08/14 21:52:56.750000  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:14 01:52:56', 'subsec': b'750000', 'offset': b

({'datetime': b'2018:08:14 01:52:56',
  'subsec': b'750000',
  'offset': b'-10:00'},
 1534247576.75,
 <DstTzInfo 'Pacific/Rarotonga' LMT-1 day, 13:21:00 STD>)

In [31]:
# Hiking next to rock (Canon)
# test("_MG_8448.jpg",tzRar)
getCorrectTimes("_MG_8448.jpg")

Exif time data extrcted from file _MG_8448.jpg is:
{'edit': {'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}, 'orig': {'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}, 'digi': {'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}
System time data extrcted from file _MG_8448.jpg is:
{'atime': 1643866333.4693193, 'mtime': 1534686434.0, 'ctime': 1643363743.2751594}
>>Min system epoch extracted:
1534686434.0
>>Min system epoch extracted in Sydney time:
2018/08/19 23:47:14.000000  AEST  +1000
>>Picture taken with b'Canon EOS DIGITAL REBEL XS'
>>Correct Unix Epoch:
1534690034.22
>>In Rarotonga
>>Correct time:
2018/08/19 04:47:14.220000  -10  -1000
>>Correct epoch and time Windows should show:
1534690034.22
2018/08/20 00:47:14.220000  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:19 04:47:14', 'subsec': b'220000', 'offset': 

({'datetime': b'2018:08:19 04:47:14',
  'subsec': b'220000',
  'offset': b'-10:00'},
 1534690034.22,
 <DstTzInfo 'Pacific/Rarotonga' LMT-1 day, 13:21:00 STD>)

In [32]:
# Hiking up the mountain (MotoG3)
# test("IMG_20180820_115912.jpg",tzRar)
getCorrectTimes("IMG_20180820_115912.jpg")

Exif time data extrcted from file IMG_20180820_115912.jpg is:
{'edit': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}, 'orig': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}, 'digi': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}
System time data extrcted from file IMG_20180820_115912.jpg is:
{'atime': 1643866335.3586392, 'mtime': 1534726752.0, 'ctime': 1643363760.136609}
>>Min system epoch extracted:
1534726752.0
>>Min system epoch extracted in Sydney time:
2018/08/20 10:59:12.000000  AEST  +1000
>>Picture taken with b'MotoG3'
>>In Rarotonga
>>Correct time:
2018/08/20 11:59:13.218714  -10  -1000
>>Correct epoch and time Windows should show:
1534802353.218714
2018/08/21 07:59:13.218714  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': b'-10:00'}


({'datetime': b'2018:08:20 11:59:13',
  'subsec': b'218714',
  'offset': b'-10:00'},
 1534802353.218714,
 <DstTzInfo 'Pacific/Rarotonga' LMT-1 day, 13:21:00 STD>)

In [33]:
# In Mexico Casa AA (LG G4)
# test("20180109_233139.jpg",tzMex)
getCorrectTimes("20180109_233139.jpg")

Exif time data extrcted from file 20180109_233139.jpg is:
{'edit': {'datetime': b'2018:01:09 23:31:39', 'subsec': b'853702', 'offset': None}, 'orig': {'datetime': b'2018:01:09 23:31:39', 'subsec': b'853702', 'offset': None}, 'digi': {'datetime': b'2018:01:09 23:31:39', 'subsec': b'853702', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:01:09 23:31:39', 'subsec': b'853702', 'offset': None}
System time data extrcted from file 20180109_233139.jpg is:
{'atime': 1643866336.9784458, 'mtime': 1515562301.95, 'ctime': 1643518483.3400853}
>>Min system epoch extracted:
1515562301.95
>>Min system epoch extracted in Sydney time:
2018/01/10 16:31:41.950000  AEDT  +1100
>>File 20180109_233139.jpg was not taken with Canon or Moto G3


UnboundLocalError: local variable 'camEpoch' referenced before assignment

In [34]:
# In Australia now (Moto g8+)
# test("IMG_20220123_230208809.jpg",tzSyd)
getCorrectTimes("IMG_20220123_230208809.jpg")

Exif time data extrcted from file IMG_20220123_230208809.jpg is:
{'edit': {'datetime': b'2022:01:23 23:02:10', 'subsec': b'812121', 'offset': None}, 'orig': {'datetime': b'2022:01:23 23:02:10', 'subsec': b'812121', 'offset': None}, 'digi': {'datetime': b'2022:01:23 23:02:10', 'subsec': b'812121', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2022:01:23 23:02:10', 'subsec': b'812121', 'offset': None}
System time data extrcted from file IMG_20220123_230208809.jpg is:
{'atime': 1643866437.4002967, 'mtime': 1642939330.0, 'ctime': 1643509065.7425287}
>>Min system epoch extracted:
1642939330.0
>>Min system epoch extracted in Sydney time:
2022/01/23 23:02:10.000000  AEDT  +1100
>>File IMG_20220123_230208809.jpg was not taken with Canon or Moto G3


UnboundLocalError: local variable 'camEpoch' referenced before assignment

In [35]:
# In Australia, darks gaby (Huawei)
# test("IMG_20190824_220407.jpg",tzSyd)
getCorrectTimes("IMG_20190824_220407.jpg")

Exif time data extrcted from file IMG_20190824_220407.jpg is:
{'edit': {'datetime': b'2019:08:24 22:04:08', 'subsec': b'632867', 'offset': None}, 'orig': {'datetime': b'2019:08:24 22:04:08', 'subsec': b'632867', 'offset': None}, 'digi': {'datetime': b'2019:08:24 22:04:08', 'subsec': b'632867', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2019:08:24 22:04:08', 'subsec': b'632867', 'offset': None}
System time data extrcted from file IMG_20190824_220407.jpg is:
{'atime': 1643866447.6113284, 'mtime': 1566644648.0, 'ctime': 1643508150.4212909}
>>Min system epoch extracted:
1566644648.0
>>Min system epoch extracted in Sydney time:
2019/08/24 21:04:08.000000  AEST  +1000
>>File IMG_20190824_220407.jpg was not taken with Canon or Moto G3


UnboundLocalError: local variable 'camEpoch' referenced before assignment

In [36]:
#Three consecutive images at the beach
print("-----------pic 1---------------")
getCorrectTimes("IMG_8230.JPG")
print("-----------pic 2---------------")
getCorrectTimes("IMG_20180814_143710.jpg")
print("-----------pic 2---------------")
getCorrectTimes("IMG_8231.JPG")

-----------pic 1---------------
Exif time data extrcted from file IMG_8230.JPG is:
{'edit': {'datetime': b'2018:08:15 10:19:06', 'subsec': b'03', 'offset': None}, 'orig': {'datetime': b'2018:08:15 10:19:06', 'subsec': b'03', 'offset': None}, 'digi': {'datetime': b'2018:08:15 10:19:06', 'subsec': b'03', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:15 10:19:06', 'subsec': b'03', 'offset': None}
System time data extrcted from file IMG_8230.JPG is:
{'atime': 1643866486.059953, 'mtime': 1534288746.0, 'ctime': 1643706066.50972}
>>Min system epoch extracted:
1534288746.0
>>Min system epoch extracted in Sydney time:
2018/08/15 09:19:06.000000  AEST  +1000
>>Picture taken with b'Canon EOS DIGITAL REBEL XS'
>>Correct Unix Epoch:
1534292346.03
>>In Rarotonga
>>Correct time:
2018/08/14 14:19:06.030000  -10  -1000
>>Correct epoch and time Windows should show:
1534292346.03
2018/08/15 10:19:06.030000  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:14 14:19:06', 's

({'datetime': b'2018:08:14 14:50:30',
  'subsec': b'030000',
  'offset': b'-10:00'},
 1534294230.03,
 <DstTzInfo 'Pacific/Rarotonga' LMT-1 day, 13:21:00 STD>)

## Testing

In [37]:
import random
todayEpoch = tzSyd.localize(datetime.datetime.today()).timestamp()
ramEpoch = tzMex.localize(datetime.datetime(1987,5,21)).timestamp()
depSydEpoch = tzSyd.localize(datetime.datetime.strptime(depSyd,"%Y:%m:%d %H:%M:%S")).timestamp()
arrSydEpoch = tzSyd.localize(datetime.datetime.strptime(arrSyd,"%Y:%m:%d %H:%M:%S")).timestamp()
print(ramEpoch,todayEpoch)
randomEpoch = random.uniform(ramEpoch,todayEpoch)
randomRaroEpoch = random.uniform(depSydEpoch,arrSydEpoch)
print(randomEpoch)
print(randomRaroEpoch)
print(epochToHuman(randomEpoch,tzMex))
print(epochToHuman(randomRaroEpoch,tzRar))


548575200.0 1643866500.535465
1642941852.7557962
1534216210.3571317
2022/01/23 06:44:12.755796  CST  -0600
2018/08/13 17:10:10.357132  -10  -1000


# Playground

In [43]:
# atime = 1534249976
# mtime = 1534243976
# testTime = 1534730353.218714
testTime = minSysEpoch("copies/IMG_20180820_115912.jpg")
print("testTime = " , testTime)
sysExifTime = epochToExif(testTime,tzSyd)
print("sysExifTime = " , sysExifTime)
sysAESTOffset = getOffset(testTime,tzSyd)
print(sysAESTOffset)
# addExifOffset("copies/IMG_20220123_230208809.jpg",sysAESTOffset)
exifTime = minExifTime("copies/IMG_20180820_115912.jpg")
print("exifTime = " , exifTime)
# os.utime("sad.jpg", (testTime, testTime))
# os.path.getmtime("sad.jpg")
# fixExifTimeData("copies/IMG_20220123_230208809.jpg",exifTime)
# changeFileDatetimes("copies/IMG_20220123_230208809.jpg",testTime)
print("-----testing-----")
# display(piexif.load("copies/IMG_20220123_230208809.jpg"))
# test("_MG_8448.jpg",tzRar)

System time data extrcted from file copies/IMG_20180820_115912.jpg is:
{'atime': 1643866593.4985135, 'mtime': 1534726752.0, 'ctime': 1643596485.2867842}
testTime =  1534726752.0
sysExifTime =  {'datetime': b'2018:08:20 10:59:12', 'subsec': b'000000', 'offset': b'+10:00'}
b'+10:00'
Exif time data extrcted from file copies/IMG_20180820_115912.jpg is:
{'edit': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}, 'orig': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}, 'digi': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}}
exifTime =  {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}
-----testing-----


In [46]:
print("--------------exif times -----------------")
canonTimeExif = minExifTime("IMG_20180820_115912 - Copy.jpg")
print(canonTimeExif)
sydExif = exifToEpoch(canonTimeExif,tzSyd)
print(sydExif)
print(epochToHuman(sydExif,tzRar))
print(epochToHuman(sydExif,tzSyd))
rarExif = exifToEpoch(canonTimeExif,tzRar)
print(rarExif)
print(epochToHuman(rarExif,tzRar))

print("--------------------system times--------------")
canonTimeSys = minSysEpoch("IMG_8137.JPG")
print(canonTimeSys)
print(epochToHuman(canonTimeSys,tzRar))

print("-------------testing------------")
# fixFileTimeMetadata("IMG_20180820_115912 - Copy.jpg")

--------------exif times -----------------
Exif time data extrcted from file IMG_20180820_115912 - Copy.jpg is:
{'edit': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}, 'orig': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}, 'digi': {'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}}
{'datetime': b'2018:08:20 11:59:13', 'subsec': b'218714', 'offset': None}
1534730353.218714
2018/08/19 15:59:13.218714  -10  -1000
2018/08/20 11:59:13.218714  AEST  +1000
1534802353.218714
2018/08/20 11:59:13.218714  -10  -1000
--------------------system times--------------
System time data extrcted from file IMG_8137.JPG is:
{'atime': 1643864850.1451693, 'mtime': 1534187122.0, 'ctime': 1643585902.48307}
1534187122.0
2018/08/13 09:05:22.000000  -10  -1000
-------------testing------------


In [51]:
aPath = r"F:\Pictures\Fotos\2018\08 Rarotonga"
os.chdir(aPath)
print(os.getcwd())
anotherPath = r".\copies"
print(anotherPath)
# thePath = aPath.replace(r"\\",r"/")
# print(thePath)
display(os.listdir(anotherPath))
os.chdir(anotherPath)
getCorrectTimes('_MG_8448 - foldertest.JPG')
#fixFolderTimeData(aPath,tzSyd)
os.chdir(aPath)

F:\Pictures\Fotos\2018\08 Rarotonga
.\copies


['20180109_233139.jpg',
 'IMG_20180817_111528.jpg',
 'IMG_20180820_115912.jpg',
 'IMG_8137.JPG',
 'IMG_8144.JPG',
 '_MG_8448 - foldertest.JPG',
 '_MG_8448.JPG',
 '_MG_8453.JPG']

Exif time data extrcted from file _MG_8448 - foldertest.JPG is:
{'edit': {'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}, 'orig': {'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}, 'digi': {'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}}
>>Min exif time extracted:
{'datetime': b'2018:08:20 00:47:14', 'subsec': b'22', 'offset': None}
System time data extrcted from file _MG_8448 - foldertest.JPG is:
{'atime': 1643866736.1214826, 'mtime': 1534686434.0, 'ctime': 1643862697.2419796}
>>Min system epoch extracted:
1534686434.0
>>Min system epoch extracted in Sydney time:
2018/08/19 23:47:14.000000  AEST  +1000
>>Picture taken with b'Canon EOS DIGITAL REBEL XS'
>>Correct Unix Epoch:
1534690034.22
>>In Rarotonga
>>Correct time:
2018/08/19 04:47:14.220000  -10  -1000
>>Correct epoch and time Windows should show:
1534690034.22
2018/08/20 00:47:14.220000  AEST  +1000
>>Correct exif time data:
{'datetime': b'2018:08:19 04:47:14', 'subs